In [0]:
!pip install torch
!pip install torchtext
!python -m spacy download en


# K80 gpu for 12 hours
import torch
from torch import nn, optim
from torchtext import data, datasets

print('GPU:', torch.cuda.is_available())

torch.manual_seed(123)


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')

GPU: True


In [0]:


TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [0]:
print('len of train data:', len(train_data))
print('len of test data:', len(test_data))

len of train data: 25000
len of test data: 25000


In [0]:
print(train_data.examples[15].text)
print(train_data.examples[15].label)

['Well', 'when', 'watching', 'this', 'film', 'late', 'one', 'night', 'I', 'was', 'simple', 'amazed', 'by', 'it', "'s", 'greatness', '.', 'Fantastic', 'script', ',', 'great', 'acting', ',', 'costumes', 'and', 'special', 'effects', ',', 'and', 'the', 'plot', 'twists', ',', 'wow', '!', '!', 'In', 'fact', 'if', 'you', 'can', 'see', 'the', 'ending', 'coming', 'you', 'should', 'become', 'a', 'writer', 'yourself.<br', '/><br', '/>Great', ',', 'I', 'would', 'recommend', 'this', 'film', 'to', 'anyone', ',', 'especially', 'if', 'I', 'don;t', 'like', 'them', 'much.<br', '/><br', '/>Terrific']
pos


In [0]:
# word2vec, glove
TEXT.build_vocab(train_data, max_size=10000, vectors='glove.6B.100d')
LABEL.build_vocab(train_data)


batchsz = 30
device = torch.device('cuda')
train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data),
    batch_size = batchsz,
    device=device
)

In [0]:
class RNN(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        """
        """
        super(RNN, self).__init__()
        
        # [0-10001] => [100]
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # [100] => [256]
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, 
                           bidirectional=True, dropout=0.5)
        # [256*2] => [1]
        self.fc = nn.Linear(hidden_dim*2, 1)
        self.dropout = nn.Dropout(0.5)
        
        
    def forward(self, x):
        """
        x: [seq_len, b] vs [b, 3, 28, 28]
        """
        # [seq, b, 1] => [seq, b, 100]
        embedding = self.dropout(self.embedding(x))
        
        # output: [seq, b, hid_dim*2]
        # hidden/h: [num_layers*2, b, hid_dim]
        # cell/c: [num_layers*2, b, hid_di]
        output, (hidden, cell) = self.rnn(embedding)
        
        # [num_layers*2, b, hid_dim] => 2 of [b, hid_dim] => [b, hid_dim*2]
        hidden = torch.cat([hidden[-2], hidden[-1]], dim=1)
        
        # [b, hid_dim*2] => [b, 1]
        hidden = self.dropout(hidden)
        out = self.fc(hidden)
        
        return out

In [0]:
rnn = RNN(len(TEXT.vocab), 100, 256)

pretrained_embedding = TEXT.vocab.vectors
print('pretrained_embedding:', pretrained_embedding.shape)
rnn.embedding.weight.data.copy_(pretrained_embedding)
print('embedding layer inited.')

optimizer = optim.Adam(rnn.parameters(), lr=1e-3)
criteon = nn.BCEWithLogitsLoss().to(device)
rnn.to(device)


pretrained_embedding: torch.Size([10002, 100])
embedding layer inited.


RNN(
  (embedding): Embedding(10002, 100)
  (rnn): LSTM(100, 256, num_layers=2, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.5)
)

In [0]:
import numpy as np

def binary_acc(preds, y):
    """
    get accuracy
    """
    preds = torch.round(torch.sigmoid(preds))
    correct = torch.eq(preds, y).float()
    acc = correct.sum() / len(correct)
    return acc

def train(rnn, iterator, optimizer, criteon):
    
    avg_acc = []
    rnn.train()
    
    for i, batch in enumerate(iterator):
        
        # [seq, b] => [b, 1] => [b]
        pred = rnn(batch.text).squeeze(1)
        # 
        loss = criteon(pred, batch.label)
        acc = binary_acc(pred, batch.label).item()
        avg_acc.append(acc)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i%10 == 0:
            print(i, acc)
        
    avg_acc = np.array(avg_acc).mean()
    print('avg acc:', avg_acc)
    
    
def eval(rnn, iterator, criteon):
    
    avg_acc = []
    
    rnn.eval()
    
    with torch.no_grad():
        for batch in iterator:

            # [b, 1] => [b]
            pred = rnn(batch.text).squeeze(1)

            #
            loss = criteon(pred, batch.label)

            acc = binary_acc(pred, batch.label).item()
            avg_acc.append(acc)
        
    avg_acc = np.array(avg_acc).mean()
    
    print('>>test:', avg_acc)
        
    
    

In [0]:
for epoch in range(10):
    
    eval(rnn, test_iterator, criteon)
    train(rnn, train_iterator, optimizer, criteon)

>>test: 0.4997602136050769
0 0.46666669845581055
10 0.40000003576278687
20 0.5
30 0.5
40 0.4333333671092987
50 0.5333333611488342
60 0.6000000238418579
70 0.5666667222976685
80 0.40000003576278687
90 0.36666667461395264
100 0.5333333611488342
110 0.6666666865348816
120 0.7333333492279053
130 0.4333333671092987
140 0.6000000238418579
150 0.5333333611488342
160 0.5
170 0.46666669845581055
180 0.6333333849906921
190 0.6666666865348816
200 0.40000003576278687
210 0.46666669845581055
220 0.5666667222976685
230 0.5
240 0.7333333492279053
250 0.6666666865348816
260 0.6333333849906921
270 0.6666666865348816
280 0.6000000238418579
290 0.7666667103767395
300 0.6000000238418579
310 0.5666667222976685
320 0.8333333730697632
330 0.6333333849906921
340 0.6000000238418579
350 0.6333333849906921
360 0.6333333849906921
370 0.7000000476837158
380 0.7333333492279053
390 0.6000000238418579
400 0.7666667103767395
410 0.7333333492279053
420 0.8000000715255737
430 0.6666666865348816
440 0.7666667103767395
45